In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from tqdm import tqdm
import seaborn as sns
import pickle 

import torch
import cv2
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
from torchvision.transforms import ToTensor
from sklearn.preprocessing import Normalizer
from PIL import Image
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


In [ ]:
data=np.load("100-Bollywood_celebrity-faces-embeddings.npz")

In [ ]:
emdTrainX, trainy, emdTestX, test=data["arr_0"],data["arr_1"],data["arr_2"],data["arr_3"]

In [ ]:
img_embeddings=np.concatenate((emdTrainX,emdTestX),axis=0)
scaler = StandardScaler()
img_embeddings=scaler.fit_transform(img_embeddings)

In [ ]:
print(img_embeddings.shape)

In [ ]:
_ = np.random.shuffle(img_embeddings)# to shuffle the image embeddings
d={"im_id":list(range(img_embeddings.shape[0]))}

In [ ]:
data=pd.DataFrame(d)

In [ ]:
data["emb"]=img_embeddings.tolist()

In [ ]:
for i in range(len(data)):
    data["emb"][i]=np.array(data["emb"][i])

## Clustering

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.metrics import silhouette_score
import hdbscan

In [ ]:
X=img_embeddings
hdbscan = hdbscan.HDBSCAN(min_cluster_size=12)
hdbscan.fit(X)
hdbscan_predictions = hdbscan.labels_
data['Cluster_DBSCAN_OPT'] = hdbscan_predictions

In [ ]:
np.unique(dbscan_predictions)

In [ ]:
a=list(dbscan_predictions)
a.count(-1)

In [ ]:
#Silhoutte_score without outliers
silhouettescore = silhouette_score(np.stack(data["emb"][dbscan_predictions!=-1]), dbscan_predictions[dbscan_predictions!=-1])
print(silhouettescore)

In [ ]:
X_train = data[data['Cluster_DBSCAN_OPT'] != -1][["emb"]].reset_index(drop=True)
y_train = data[data['Cluster_DBSCAN_OPT'] != -1][['Cluster_DBSCAN_OPT']].reset_index(drop=True)
X_test = data[data['Cluster_DBSCAN_OPT'] == -1][["emb"]].reset_index(drop=True)

In [ ]:
KNC = KNeighborsClassifier(n_neighbors=5)
KNC.fit(np.stack(X_train["emb"].values), y_train.values.ravel())
KNC_predictions = KNC.predict(np.stack(X_test["emb"].values))

In [ ]:
data['Cluster_Hybrid'] = data['Cluster_DBSCAN_OPT']
data.loc[data['Cluster_DBSCAN_OPT'] == -1, 'Cluster_Hybrid'] = KNC_predictions

In [ ]:
silhouettescore = silhouette_score(np.stack(data["emb"]), dbscan_predictions)

In [ ]:
silhouettescore